In [77]:
import warnings
warnings.filterwarnings('ignore')
from pymongo import MongoClient
import pprint

import copy
import pandas as pd

import requests

from bs4 import BeautifulSoup
import json
import re
import numpy as np

In [15]:
steamspy_url = 'https://steamspy.com/api.php?request=all'
payload = {'action': 'parse', 'format': 'json'}
r = requests.post(steamspy_url)

In [138]:
client = MongoClient('localhost', 27017)
db = client.steamspy
test = db.test

In [18]:
for item in r.json().values():
    test.insert(item)

In [19]:
data = pd.DataFrame(list(test.find()))

In [48]:
for idx in range(data.shape[0]):
    owners = data.iloc[idx]['owners'].split('..')
    for i in range(len(owners)):
        owners[i] = owners[i].replace(',','')
        owners[i] = owners[i].replace(' ','')
        owners[i] = int(owners[i])
    data.at[idx, 'llOwners'] = owners[0]
    data.at[idx, 'ulOwners'] = owners[1]

In [54]:
data = data.drop(columns = ['score_rank', 'owners', '_id'])

In [27]:
#Params: genre(string) represents the genre you want to get data of
#        collection(database collection) represents database to store the genre data in
#This function gets the data of a certain genre from the steamspy website and puts it in the given collection
def storeGenre(genre, collection):
    steamspy_url = 'https://steamspy.com/api.php?request=genre&genre={}'.format(genre)
    r = requests.post(steamspy_url)
    for item in r.json().values():
        collection.insert(item)

In [ ]:
#Params: tag(string) represents the tag you want to get data of
#        collection(database collection) represents database to store the gtag data in
#This function gets the data of a certain tag from the steamspy website and puts it in the given collection
def storeTag(tag, collection):
    steamspy_url = 'https://steamspy.com/api.php?request=tag&tag={}'.format(genre)
    r = requests.post(steamspy_url)
    for item in r.json().values():
        collection.insert(item)

In [28]:
action = db.action

In [29]:
storeGenre('Action', action)

In [30]:
earlyaccess = db.earlyaccess
storeGenre('Early+Access', earlyaccess)

In [126]:
dataAction = pd.DataFrame(list(action.find()))

In [191]:
data['Genres'] = np.empty((len(df), 0)).tolist()
data['Tags'] = np.empty((len(df), 0)).tolist()

In [188]:
#Params: df(dataframe) represents the data to append to
#        genreDB(database) represents the genre database that you are adding
#        genre(string) the genre you are appending
#This function appends to the Genre column of df if the game is in the certain genre
def appendGenre(df, genreDB, genre):
    genreData = pd.DataFrame(list(genreDB.find()))
    genreData = genreData.drop(columns = ['score_rank', 'owners', '_id'])
    df['{}'.format(genre)] = data.appid.isin(genreData.appid).astype(int)
    for idx in range(df.shape[0]):
        if df.iloc[idx]['{}'.format(genre)] == 1:
            df.at[idx, 'Genres'].append('{}'.format(genre))
    df.drop(columns = '{}'.format(genre), axis=1, inplace = True)
    
#     df = pd.merge(df, genreData, on=['appid','average_2weeks','average_forever','developer',
#                                    'discount', 'initialprice','median_2weeks','median_forever',
#                                    'name', 'negative', 'positive', 'price', 'publisher',
#                                    'userscore'], how='left', indicator='{}'.format(genre))

In [182]:
#Params: df(dataframe) represents the data to append to
#        tagDB(database) represents the tag database that you are adding
#        tag(string) the tag you are appending
#This function appends to the Tags column of df if the game includes a certain tag
def appendTag(df, tagDB, tag):
    tagData = pd.DataFrame(list(tagDB.find()))
    tagData = tagData.drop(columns = ['score_rank', 'owners', '_id'])
    df['{}'.format(tag)] = data.appid.isin(tagData.appid).astype(int)
    for idx in range(df.shape[0]):
        if df.iloc[idx]['{}'.format(tag)] == 1:
            df.at[idx, 'Tags'].append('{}'.format(tag))
    df.drop(columns = '{}'.format(tag), axis=1, inplace = True)

In [139]:
indie = db.indie

In [193]:
appendGenre(data, action, 'Action')

In [140]:
storeGenre('Indie', indie)

In [184]:
appendTag(data, indie, 'Indie')

In [176]:
#data = data.drop(columns = 'Indie')
data = data.drop(columns = 'Action')

In [194]:
data.head(20)

,appid,average_2weeks,average_forever,developer,discount,initialprice,median_2weeks,median_forever,name,negative,positive,price,publisher,userscore,ulOwners,llOwners,Genres,Tags
0,570,1849,31812,Valve,0,0,825,1334,Dota 2,172648,1012914,0,Valve,0,200000000,100000000,[Action],[]
1,578080,713,20720,PUBG Corporation,50,2999,194,9457,PLAYERUNKNOWN'S BATTLEGROUNDS,577080,666523,1499,PUBG Corporation,0,100000000,50000000,[Action],[]
2,730,908,25215,"Valve, Hidden Path Entertainment",0,0,282,8021,Counter-Strike: Global Offensive,467027,3266572,0,Valve,0,100000000,50000000,[Action],[]
3,440,1115,8960,Valve,0,0,252,458,Team Fortress 2,36970,585130,0,Valve,0,100000000,50000000,[Action],[]
4,304930,2028,3740,Smartly Dressed Games,0,0,295,341,Unturned,33274,319833,0,Smartly Dressed Games,0,50000000,20000000,[Action],[]
5,230410,1155,6037,Digital Extremes,0,0,309,394,Warframe,28108,293762,0,Digital Extremes,0,50000000,20000000,[Action],[]
6,271590,700,8750,Rockstar North,50,2999,273,5130,Grand Theft Auto V,157356,500821,1499,Rockstar Games,0,50000000,20000000,[Action],[]
7,10,815,9671,Valve,90,999,925,287,Counter-Strike,3739,143404,99,Valve,0,50000000,20000000,[Action],[]
8,291550,490,1284,Blue Mammoth Games,0,0,121,135,Brawlhalla,18645,104110,0,Ubisoft,0,50000000,20000000,[Action],[]
9,240,417,7125,Valve,90,999,150,476,Counter-Strike: Source,3981,92419,99,Valve,0,50000000,20000000,[Action],[]
